In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import glob
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci
from PIL import Image
from scipy import ndimage
import cv2

In [3]:
data_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/data/groundTruth/'
save_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/groundTruth/'

epsilon = 0.0001 #just to round down
DEFAULT_IMG_SIZE = (481,481,3) #defaul image size

### Read Images

In [5]:
data_types = ['val'] #, 'train']

image_names, names = [], []

for data_type in data_types:
    image_path = data_path + data_type + '/'
    names_path = save_path + data_type + '/'
    
    len_path = len(image_path)
    image_files = glob.glob(image_path + '*.jpg')
    image_filenames = [x[len_path:-4] for x in image_files]

    #load images
    for filename in image_filenames:
        file = image_path + filename + '.jpg'

        #append image and groundtruths
        image_names.append(file)
        names.append(names_path + filename + '_&&&.png')

print(len(image_names), len(names))

300 300


In [10]:
index = 0
len_images = len(names)
for image_name, name in zip(image_names, names):
    index += 1

    black_image = np.zeros(DEFAULT_IMG_SIZE,dtype=np.uint8)
    image = io.imread(image_name)
    
    prefix_name = name[:-4]
    extension_name = name[-4:]

    if(image.shape[0] == 321):
        black_image[80:401,:,:] = image[:,:,:]
    else:
        black_image[:,80:401,:] = image[:,:,:]
        
    flip_types = ['noflip', 'fliplr']
    
    for flip_type in flip_types:
        if flip_type == 'fliplr':
            flip_image = np.fliplr(image)
            flip_black = np.fliplr(black_image)
        elif flip_type == 'flipud':
            flip_image = np.flipud(image)
            flip_black = np.flipud(black_image)
        else:
            flip_black = black_image
            flip_image = image
        
        #save image 0
        save_name = prefix_name.replace("&&&", ('_' + flip_type + '_0'))
        save_name = (save_name + extension_name)
        io.imsave(save_name, flip_black)

        #rotate image and save
        for angle in range(30, 360, 30):
            rot_image = ndimage.rotate(flip_image, angle)
            res_image = cv2.resize(rot_image, dsize=DEFAULT_IMG_SIZE[:2], interpolation=cv2.INTER_CUBIC)

            save_name = prefix_name.replace("&&&", ('_' + flip_type + '_' + str(angle)))
            save_name = (save_name + extension_name)
            io.imsave(save_name, res_image)
    
    print(index, '/', len_images, end='')
    print('\r', end='')

print('OK!')

OK! / 300
